In [ ]:
%load_ext kamu

In [ ]:
%import_dataset us.cityofnewyork.data.zipcode-boundaries --alias zipcodes

In [ ]:
zipcodes.printSchema()

In [ ]:
%%sql
select * from zipcodes limit 5

In [ ]:
%%sql -o population_density
with with_area as (
    select
        geometry,
        modzcta,
        pop_est as population,
        st_area(st_geomfromgeojson(geometry)) as area
    from zipcodes
)
select
    geometry,
    modzcta,
    population,
    area,
    population / area as density
from with_area

In [ ]:
%%local
import json

# For every row we first combine GeoJson geometry with other columns into a Feature object
# Then we combine all Features into a FeatureCollection
def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]
    
    return {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": json.loads(row[geom]),
                "properties": {p: row[p] for p in props}
            }
            for _, row in df.iterrows()
        ]
    }

In [ ]:
%%local
gj = df_to_geojson(population_density)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

num_max = population_density.density.max()
stops = [i * num_max / 4 for i in range(4)]

viz = ChoroplethViz(
    gj, style='mapbox://styles/mapbox/dark-v10', center=(-74.068266, 40.7019673), zoom=9,
    access_token=token,
    color_property='density',
    color_stops=create_color_stops(stops, colors='YlOrRd'),
    color_function_type='interpolate',
    line_stroke='solid',
    line_width=0.3,
    line_color='rgb(128,0,38)',
    opacity=0.8,)
viz.show()